## Set up

In [1]:
import pickle
from dataclasses import dataclass
from pathlib import Path
from typing import ClassVar

import keras as tfk
import numpy as np
import tensorflow as tf
from keras import layers as tfkl

2024-11-16 07:41:18.487093: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 07:41:18.487347: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 07:41:18.491134: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 07:41:18.522794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 07:41:19.196575: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:
print(f"Keras version is {tfk.__version__}")

Keras version is 3.3.3


In [3]:
AUGMENTED_DIR = Path().absolute().parent / "data" / "augmented"
OUTPUT_DIR = Path().absolute().parent / "submission"
# If the data is not there, then we're running in Kaggle
if not AUGMENTED_DIR.exists():
    AUGMENTED_DIR = Path("/kaggle/input/an2dl-homework-1-augmented/augmented")
    OUTPUT_DIR = Path()

TRAIN_TFDS_DIR = AUGMENTED_DIR / "train"
VAL_TFDS_DIR = AUGMENTED_DIR / "val"

print(TRAIN_TFDS_DIR)
print(VAL_TFDS_DIR)
print(OUTPUT_DIR)

/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-1/data/augmented/train
/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-1/data/augmented/val
/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-1/submission


In [4]:
SEED = 42
KERAS_VERBOSITY = 2
BATCH_SIZE = 128
LOSS = "categorical_crossentropy"
METRICS = ["accuracy", "precision", "recall", "f1_score"]

In [5]:
tfk.utils.set_random_seed(SEED)

In [ ]:
@dataclass
class Hyperparameters:
    # Model
    global_pooling_type = "avg"
    layer_units = (
        128,
        64,
    )
    activation = "silu"
    class_weight: ClassVar[dict[int, float]] = {
        0: 1.00,
        1: 1.00,
        2: 1.00,
        3: 1.00,
        4: 1.25,
        5: 1.25,
        6: 1.25,
        7: 1.00,
    }
    ## Regularisation
    regularisations = (
        (3e-3, 0.0),
        (0.0, 1e-2),
    )
    dropout = 0.7
    noise_std = 0.075
    # Training
    optimiser = tfk.optimizers.AdamW
    learning_rate = 1e-4
    epochs = 200
    ## Early stopping parameters
    es_patience = 20
    es_min_delta = 1e-3
    ## Learning rate schedule
    lr_patience = 5
    lr_min_delta = 1e-2
    lr_decay_factor = 0.1
    min_lr = 1e-7

hp = Hyperparameters()

### Load

In [7]:
train_dataset = tf.data.Dataset.load(str(TRAIN_TFDS_DIR)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.load(str(VAL_TFDS_DIR)).batch(BATCH_SIZE)

train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 8), dtype=tf.uint8, name=None))>

## Model

In [ ]:
def build(hp: Hyperparameters) -> tfk.Model:
    inputs = tfkl.Input((96, 96, 3))
    x = inputs

    x = tfkl.Rescaling(1.0/255.0)(x)
    x = tfkl.GaussianNoise(hp.noise_std)(x)
    x = tfkl.Rescaling(255.0)(x)

    # ImageNet images are 224x224 so we need to resize to use the
    # pre-trained backbone feature extractor
    x = tfkl.Resizing(224, 224)(x)

    # TODO Investigate changing `input_shape` to actual image shape
    # instead of resizing
    feature_extractor = tfk.applications.MobileNetV3Large(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet",
        pooling=hp.global_pooling_type,
    )
    feature_extractor.trainable = False
    x = feature_extractor(x)

    x = tfkl.BatchNormalization()(x)

    for units, regularisation in zip(hp.layer_units, hp.regularisations):
        x = tfkl.Dense(
            units,
            activation=hp.activation,
            kernel_regularizer=tfk.regularizers.L1L2(*regularisation),
        )(x)

    x = tfkl.Dropout(hp.dropout)(x)

    # Classification problem has 8 output classes
    # so the final layer has 8 neurons
    # with a softmax activation
    outputs = tfkl.Dense(8, activation="softmax")(x)

    return tfk.Model(inputs, outputs)

In [9]:
model = build(hp)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_resnet_v2             │ (None, 1536)           │    54,336,736 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_203         │ (None, 1536)           │         6,144 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,548,392 (208.09 MB)

 Trainable params: 208,584 (814.78 KB)

 Non-trainable params: 54,339,808 (207.29 MB)

In [10]:
def fit(
    hp: Hyperparameters,
    train: tf.data.Dataset,
    val: tf.data.Dataset,
    model: tfk.Model | None = None,
) -> tuple[tfk.Model, tfk.callbacks.History]:
    model = model or build(hp)
    model.compile(
        optimizer=hp.optimiser(hp.learning_rate),
        loss=LOSS,
        metrics=METRICS,
    )
    history = model.fit(
        train.prefetch(tf.data.AUTOTUNE),
        epochs=hp.epochs,
        validation_data=val.prefetch(tf.data.AUTOTUNE),
        class_weight=hp.class_weight,
        callbacks=[
            tfk.callbacks.EarlyStopping(
                min_delta=hp.es_min_delta,
                patience=hp.es_patience,
                restore_best_weights=True,
                verbose=1,
            ),
            tfk.callbacks.ReduceLROnPlateau(
                factor=hp.lr_decay_factor,
                patience=hp.lr_patience,
                min_delta=hp.lr_min_delta,
                min_lr=hp.min_lr,
                verbose=1,
            ),
        ],
        verbose=KERAS_VERBOSITY,
    )
    return model, history

In [ ]:
model, history = fit(hp, train_dataset, val_dataset)

Epoch 1/300


## Save results

In [ ]:
model.save(OUTPUT_DIR / "model.keras")

In [ ]:
with (OUTPUT_DIR / "history.pkl").open("wb") as f:
    pickle.dump(history, f)

In [ ]:
np.save(
    OUTPUT_DIR / "train_predictions",
    model.predict(train_dataset, verbose=KERAS_VERBOSITY),
)
np.save(
    OUTPUT_DIR / "val_predictions",
    model.predict(val_dataset, verbose=KERAS_VERBOSITY),
)